In [16]:
pip install py2neo requests



   ---------- ----------------------------- 1/4 [pansi]
   -------------------- ------------------- 2/4 [interchange]
   ------------------------------ --------- 3/4 [py2neo]
   ------------------------------ --------- 3/4 [py2neo]
   ------------------------------ --------- 3/4 [py2neo]
   ------------------------------ --------- 3/4 [py2neo]
   ------------------------------ --------- 3/4 [py2neo]
   ---------------------------------------- 4/4 [py2neo]



In [22]:
# Étape 2 : Import des modules nécessaires
from math import radians, sin, cos, acos
from py2neo import Graph, Node, Relationship
import requests


In [23]:
# Étape 3 : Définition de la position de Fès
fes_lat = 34.0331
fes_lon = -4.9998


In [24]:
# Étape 4 : Interroger Overpass API pour les hôtels autour de Fès
def query_overpass(latitude, longitude, radius=4000):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    (
      node["tourism"="hotel"](around:{radius},{latitude},{longitude});
      way["tourism"="hotel"](around:{radius},{latitude},{longitude});
      relation["tourism"="hotel"](around:{radius},{latitude},{longitude});
    );
    out center;
    """
    response = requests.get(overpass_url, params={'data': overpass_query})
    return response.json()


In [25]:
# Étape 5 : Calcul de distance (Haversine simplifié)
def calculate_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    return acos(sin(lat1)*sin(lat2) + cos(lat1)*cos(lat2)*cos(lon2-lon1)) * 6371  # km


In [26]:
# Étape 6 : Traitement des résultats en liste de dictionnaires
def process_hotels(results, ref_lat, ref_lon):
    hotels = []
    for element in results.get("elements", []):
        name = element.get("tags", {}).get("name", "Hôtel inconnu")
        lat = element.get("lat") or element.get("center", {}).get("lat")
        lon = element.get("lon") or element.get("center", {}).get("lon")
        if lat and lon:
            distance = calculate_distance(ref_lat, ref_lon, float(lat), float(lon))
            hotels.append({
                "name": name,
                "latitude": float(lat),
                "longitude": float(lon),
                "distance": round(distance, 3)
            })
    return hotels


In [38]:
# Étape 7 : Connexion à Neo4j et insertion des données
# Remplacez les identifiants ici avec les vôtres
graph = Graph("bolt://localhost:7687", auth=("neo4j", "elouazzani"))

def store_in_neo4j(hotels, ref_lat, ref_lon):
    graph.delete_all()  # Nettoyage de la base
    
    location_node = Node("Location", name="Fès", latitude=ref_lat, longitude=ref_lon)
    graph.create(location_node)

    for hotel in hotels:
        hotel_node = Node("Hotel",
                          name=hotel["name"],
                          latitude=hotel["latitude"],
                          longitude=hotel["longitude"],
                          distance=hotel["distance"])
        graph.create(hotel_node)
        relation = Relationship(location_node, "IS_NEAR", hotel_node)
        graph.create(relation)


In [36]:
# Étape 8 : Requête pour obtenir l'hôtel le plus proche
def get_closest_hotel():
    query = """
    MATCH (l:Location {name:'Fès'})-[:IS_NEAR]->(h:Hotel)
    RETURN h.name AS name, h.distance AS distance
    ORDER BY h.distance ASC
    LIMIT 1
    """
    result = graph.run(query).data()
    return result[0] if result else None


In [37]:
# Étape 9 : Lancement du processus complet
results = query_overpass(fes_lat, fes_lon)
hotels = process_hotels(results, fes_lat, fes_lon)
store_in_neo4j(hotels, fes_lat, fes_lon)

closest = get_closest_hotel()
if closest:
    print("✅ Hôtel le plus proche de Fès :")
    print(f"🏨 {closest['name']}")
    print(f"📏 Distance : {closest['distance']} km")
else:
    print("Aucun hôtel trouvé.")


ServiceUnavailable: Cannot connect to any known routers